In [ ]:
val path = java.nio.file.FileSystems.getDefault().getPath("/Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/target/scala-2.11/histogrammodel_2.11-0.1.jar")
val x = ammonite.ops.Path(path)
interp.load.cp(x)

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Histogram

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

plotly.JupyterScala.init()

In [ ]:
implicit class DrawableHistogram[E](hist: Histogram[E]) {
    def draw() = {
        val xs = hist.elementsPresent.toSeq.sortBy(_.toString)
        val ys = xs.map(e => hist(e).toDouble)
        Bar(xs.map(_.toString), ys).plot()
    }
}

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.atomizers.Pixel
import java.io.File
import javax.imageio.ImageIO.read

def readImages(folder: File) = {
    val files = folder.isDirectory match {
        case true => folder.listFiles().toList
        case false => List(folder)
    }
    val names = files.map(_.getName.split('.')(0))
    val hists = files.map(read).map(_.toHistogram)
    names.zip(hists)
}

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.HistogramInput

In [ ]:
val imagesHists = readImages(new File("images")).toMap.mapValues(_.normalize)
val imagesAliases = imagesHists.mapValues( hist => HistogramInput(hist))

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.SubhistogramInput

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.atomizers._

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.Query

In [ ]:
val pixelsAliases = Map("blue" -> SubhistogramInput(Set(Blue)),
                       "red" -> SubhistogramInput(Set(Red, Yellow)),
                       "green" -> SubhistogramInput(Set(Green)),
                       "yellow" -> SubhistogramInput(Set(Yellow)),
                       "cyan" -> SubhistogramInput(Set(Cyan)),
                       "magenta" -> SubhistogramInput(Set(Magenta)))

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.Input

implicit val aliases = (Query.standardAliases[Pixel] ++ imagesAliases ++ pixelsAliases).asInstanceOf[Map[String, Input[Pixel]]]

In [ ]:
imagesHists.keys

In [ ]:
imagesHists("tiger")

In [21]:
val q = Query.fromString("red | yellow")(aliases)

q: Query[Pixel] = ru.hse.se.ba.danilin.paul.histogram_model.queries.Query@3ddcee55

In [22]:
val sea = imagesHists("sea")

val res = q.execute(sea).left.get

res similar sea

sea: Histogram[Pixel] = HistogramImpl(Map(BLUE -> 0.10955830416519766, YELLOW -> 0.24812603057717167, MAGENTA -> 0.24812603057717167, GREEN -> 0.01839813727782644, CYAN -> 0.24811472275025737, RED -> 0.1276767746523752))
res: Histogram[Pixel] = HistogramImpl(Map(RED -> 0.1276767746523752, YELLOW -> 0.24812603057717167))
res21_2: Double = 0.3758028052295469

In [23]:
val tiger = imagesHists("tiger")
tiger.draw

tiger: Histogram[Pixel] = HistogramImpl(Map(BLUE -> 0.019837303272548706, YELLOW -> 0.24517671224058077, MAGENTA -> 0.24517511531546007, GREEN -> 0.04098198917983423, CYAN -> 0.24483996065575733, RED -> 0.2039889193358189))
res22_1: String = "plot-442294103"

In [24]:
val res = q.execute(tiger).left.get
res.draw
tiger similar sea

res: Histogram[Pixel] = HistogramImpl(Map(RED -> 0.2039889193358189, YELLOW -> 0.24517671224058077))
res23_1: String = "plot-2016719296"
res23_2: Double = 0.9011040034145485